In [59]:
from kubernetes import client, config
import subprocess
import json 
from datetime import datetime
import threading

import csv   

fields=['timestamp', 'pod','cpu','memory', 'window']

# ['usage'] 

def getMetrics(name):
    result = subprocess.run(['kubectl', 'get', '--raw', '/apis/metrics.k8s.io/v1beta1/namespaces/sim/pods/' + name], stdout=subprocess.PIPE)
    metric = json.loads(result.stdout)
    return metric['containers'][0]

class StoppableThread(threading.Thread):
    """Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition."""

    def __init__(self):
        super(StoppableThread, self).__init__()
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

class CollectionThread(StoppableThread):

    def __init__(self, performanceFile):
        super().__init__()
        self.performanceFile = performanceFile


    def run(self):
        # Configs can be set in Configuration class directly or using helper utility
        config.load_kube_config()

        v1 = client.CoreV1Api()
        ret = v1.list_pod_for_all_namespaces(watch=False)

        with open(self.performanceFile, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
            while not self.stopped():
                timestamp = datetime.now()
                for i in ret.items:
                    if (i.metadata.namespace == "sim"):
                        m = getMetrics(i.metadata.name)
                        writer.writerow([timestamp, i.metadata.name, int(m['usage']['cpu'][:-1]), int(m['usage']['memory'][:-2]), 0])
            print("stopped")

In [60]:
import threading
import time
import os
import shutil

output = "numberOfServicesNoWork"
version = "v1"

outputDir = os.path.join(output, version)

if not os.path.exists(outputDir):
    os.mkdir(outputDir)

performanceFile = os.path.join(outputDir, 'podPerformance.csv')
resultFile = os.path.join(outputDir, 'result.jtl')

if os.path.exists(performanceFile):
    os.remove(performanceFile)

if os.path.exists(resultFile):
    os.remove(resultFile) 


collection_thread = CollectionThread(performanceFile)
collection_thread.start()


subprocess.run(['jmeter', '-n', '-t', 'jmeterRun.jmx', '-f', '-l', resultFile])

collection_thread.stop()

Creating summariser <summary>
Created the tree successfully using jmeterRun.jmx
Starting standalone test @ Thu Jul 21 14:38:26 CEST 2022 (1658407106403)
Waiting for possible Shutdown/StopTestNow/HeapDump/ThreadDump message on port 4445
summary +    101 in 00:00:04 =   28.3/s Avg:   111 Min:     0 Max:   306 Err:     0 (0.00%) Active: 8 Started: 8 Finished: 0
summary +   2157 in 00:00:30 =   71.9/s Avg:   256 Min:     0 Max:  1733 Err:     2 (0.09%) Active: 21 Started: 21 Finished: 0
summary =   2258 in 00:00:34 =   67.2/s Avg:   249 Min:     0 Max:  1733 Err:     2 (0.09%)
summary +   1855 in 00:00:30 =   61.8/s Avg:   223 Min:     0 Max: 11653 Err:    11 (0.59%) Active: 21 Started: 21 Finished: 0
summary =   4113 in 00:01:04 =   64.7/s Avg:   237 Min:     0 Max: 11653 Err:    13 (0.32%)
summary +   3234 in 00:00:30 =  107.8/s Avg:   242 Min:     0 Max: 19517 Err:    10 (0.31%) Active: 20 Started: 21 Finished: 1
summary =   7347 in 00:01:34 =   78.5/s Avg:   239 Min:     0 Max: 19517 E